# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_excel - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Function

## Get the existing excel files in "lh_bronze"

In [ ]:
# Will be joined to datasets "archive table list" and "extract table list" to exclude the non-existing files in "lh_bronze" from the loops

if "Excel" in list_technology:
    def fn_get_excel_files_in_lh_bronze(
        excel_folder_name
        , list_frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Create empty dataframe to keep the existing files in the lakehouse
            schema_exist = st.StructType([
                st.StructField('folder_name_original', st.StringType())
                , st.StructField('file_name_original', st.StringType())
                , st.StructField('folder_name', st.StringType())
                , st.StructField('file_name', st.StringType())
            ])
            sdf_exist = spark.createDataFrame([], schema_exist)

            # Collect "folder_name", "file_name"
            sql_code = f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\") AS `folder_name`
    , `file_name`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`
WHERE
    `technology`       = \"Excel\"
    AND `frequency`    IN ({str(list_frequency).replace("[", "").replace("]", "")})
    AND `is_extracted` = 1;"""
            sdf_folder_file = fn_execute_spark_sql(sql_code)[2]            

            for row in sdf_folder_file.collect():
                dict_parameter = {"technology": row.technology, "frequency": row.frequency, "folder_name": row.folder_name, "file_name": row.file_name}
                folder_name_clean = row.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")
                file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(row.file_name)[2], "_")[2]
                sdf_file          = fn_list_folder(
                    f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/{excel_folder_name}/{folder_name_clean}/{file_name_clean}.xlsx"
                    , dict_parameter
                )

                for r in sdf_file[2]:
                    new_row = spark.createDataFrame([(row.folder_name, row.file_name, folder_name_clean, r.name)], schema_exist)
                    sdf_exist = sdf_exist.union(new_row)

                sdf_exist = sdf_exist.withColumn("file_name", sf.regexp_replace("file_name", ".xlsx", ""))
            sdf_exist_count = sdf_exist.count()

            alert             = "Success"
            alert_description = f"Existing excel files (count): {sdf_exist_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_exist, sdf_exist_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Read Excel file

In [ ]:
if "Excel" in list_technology:
    def fn_read_excel_file(
        file_name
        , sheet_name
        , first_row
        , dict_parameter
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            pdf = pd.read_excel(
                io = file_name
                , sheet_name = sheet_name
                , header = first_row
                , dtype = str
            )
            pdf_count = len(pdf.index)

            alert             = "Success"
            alert_description = f"Excel file (rows): {pdf_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, pdf, pdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            pdf = pdf.head(5)
            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operation

## Extract Ecel

In [ ]:
if "Excel" in list_technology:
    def fn_extract_excel(
        technology
        , frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Rename the existing files in "lh_bronze"
            fn_rename_files_in_lh_bronze(technology, frequency, "Archive")

            # Get the existing excel files in "lh_bronze"
            sdf_exist = fn_get_excel_files_in_lh_bronze(technology, list_frequency)[2]
            if is_debug:
                print(f"sdf_exist:")
                display(sdf_exist)

            # Get extract table list
            dict_parameter = {"technology": technology, "frequency": frequency, "action": "Extract"}
            sdf_extract = fn_get_extract_table_list(dict_parameter)[2]

            # Join "existing" and "extract" files to remove the not existing from the "loop" list
            sdf_extract = sdf_extract.alias("x")\
                .join(
                    sdf_exist.alias("e")
                    , [
                        sf.col("x.folder_name") == sf.col("e.folder_name_original")
                        , sf.col("x.file_name") == sf.col("e.file_name_original")
                    ]
                    , "inner")\
                .select(sf.col("x.technology"), sf.col("x.frequency"), sf.col("x.folder_name")
                    , sf.col("x.file_name"), sf.col("x.worksheet_name"), sf.col("x.first_row"))
            if is_debug:
                print(f"sdf_extract:")
                display(sdf_extract)

            # Loop extract list
            for row_e in sdf_extract.collect():
                # Clean up "folder_name", "file_name"
                folder_name_clean = row_e.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")    
                folder_name_clean = fn_clean_accent_in_string(folder_name_clean)[2]
                file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(row_e.file_name)[2], "_")[2]
                #file_name_clean   = fn_clean_accent_in_string(file_name_clean)[2] ???
                
                # Prepare the parameters for "fn_read_excel_file()"
                excel_file_path      = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/Excel/{folder_name_clean}/{file_name_clean}.xlsx"
                sheet_name     = row_e.worksheet_name
                first_row      = int(row_e.first_row)
                dict_parameter = {"technology": row_e.technology
, "frequency": row_e.frequency
, "folder_name": row_e.folder_name
, "folder_name_clean": folder_name_clean
, "file_name": row_e.file_name
, "file_name_clean": file_name_clean
, "worksheet_name": row_e.worksheet_name}
                if is_debug:
                    print(f"file_path: {excel_file_path}")
                    print(f"sheet_name: {sheet_name}")
                    print(f"first_row: {first_row}")
                    print(f"dict_parameter: {dict_parameter}")

                # Generate column list for the current worksheet from table "lh_cfg.extract_object_excel"    
                sdf_column = fn_get_extract_column(dict_parameter)[2]
                sdf_column = sdf_column.withColumn("alias_name", sf.trim(fn_clean_accent_in_sdf_column("column_name")[2]))
                sdf_column = fn_replace_all_not_alphanumericals_in_single_column_in_sdf(sdf_column, "alias_name", "_")[2]
                if is_debug:
                    print("sdf_column:")
                    display(sdf_column)
                    
                # Extract to Pandas dataframe
                pdf_extracted = fn_read_excel_file(excel_file_path, sheet_name, first_row, dict_parameter)[2]
                if is_debug:
                    print("pdf_extracted:")
                    pdf_extracted.info(verbose = True)
                    display(pdf_extracted)

                # Convert to spark dataframe
                sdf_extracted = spark.createDataFrame(pdf_extracted)
                sdf_extracted_count = sdf_extracted.count()

                # Replace NaN and empty string with NULL in all the columns of the dataframe
                sdf_extracted = fn_replace_in_all_columns_in_sdf(sdf_extracted, "NaN", None)[2]
                sdf_extracted = fn_replace_in_all_columns_in_sdf(sdf_extracted, "", None)[2]
                sdf_extracted = eval(fn_get_select_column("sdf_extracted", sdf_column)[2])
                if is_debug:
                    print("sdf_extracted:")
                    sdf_extracted.printSchema()        
                    display(sdf_extracted)

                # Save extracted file as delta table in lh_bronze - Start
                # Replace "/" with "_" in "folder_name"
                folder_name_clean = folder_name_clean.replace("/", "_")

                # Clean up "excel_table_path" components
                technology_clean     = fn_clean_accent_in_string(row_e.technology)[2]
                technology_clean     = fn_clean_not_alphanumeric_in_string(technology_clean, "_")[2]
                worksheet_name_clean = fn_clean_accent_in_string(row_e.worksheet_name)[2]
                worksheet_name_clean = fn_clean_not_alphanumeric_in_string(worksheet_name_clean, "_")[2]
                folder_name_clean    = fn_clean_accent_in_string(folder_name_clean)[2]
                folder_name_clean    = fn_clean_not_alphanumeric_in_string(folder_name_clean, "_")[2]
                file_name_clean      = fn_clean_accent_in_string(row_e.file_name)[2]
                file_name_clean      = fn_clean_not_alphanumeric_in_string(file_name_clean, "_")[2]                
                excel_table_path     = f"{global_parameter.abfs_path_lh_bronze}/Tables/{technology_clean}_{folder_name_clean}_{file_name_clean}_{worksheet_name_clean}"
                
                dict_parameter["technology_clean"]     = technology_clean
                dict_parameter["folder_name_clean"]    = folder_name_clean
                dict_parameter["file_name_clean"]      = file_name_clean
                dict_parameter["worksheet_name_clean"] = worksheet_name_clean

                # Remove double underscore (folder_name is empty)
                excel_table_path = excel_table_path.replace(f"__", "_")

                fn_save_sdf_as_table(
                    sdf_extracted
                    , "delta"
                    , "overwrite"
                    , excel_table_path
                    , dict_parameter
                )
                # Save extracted file as delta table in lh_bronze - End

                # Insert "Success" in lh_log (for each file) - Start
                alert             = "Success"
                alert_description = f"Row count: {sdf_extracted_count}"

                pdf_extracted = pdf_extracted.head(5)

                fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)
                # Insert "Success" in lh_log (for each file) - End

                if is_debug: print("\n-------------------- Loop Extract End --------------------\n")

            # Copy to lh_bronze/Files/processed (archive) - Start
            # Create sdf_archive - Start

            # Get archive table list
            dict_parameter = {"technology": technology, "frequency": frequency, "action": "Archive"}
            sdf_archive = fn_get_extract_table_list(dict_parameter)[2]

            # Join "existing" and "archive" files to remove the not existing from the "loop" list
            sdf_archive = sdf_archive.alias("a")\
                .join(
                    sdf_exist.alias("e")
                    , [
                        sf.col("a.folder_name") == sf.col("e.folder_name_original")
                        , sf.col("a.file_name") == sf.col("e.file_name_original")
                    ]
                    , "inner")\
                .select(sf.col("a.technology"), sf.col("a.frequency"), sf.col("a.folder_name"), sf.col("a.file_name"))
            if is_debug:
                print(f"sdf_archive:")
                display(sdf_archive)
            # Create sdf_archive - End

            # Prepare dstn path components
            now_year = fn_get_now("year")[1]
            now_month = fn_get_now("month")[1]

            # Loop sdf_archive
            for row_a in sdf_archive \
                .sort(
                    sdf_archive.technology.asc()
                    , sdf_archive.frequency.asc()
                    , sdf_archive.folder_name.asc()
                    , sdf_archive.file_name.asc()
                ) \
                .collect():

                # Clean up "sdf_csv_file_table_path" components
                technology_clean  = fn_clean_accent_in_string(row_a.technology)[2]
                technology_clean  = fn_clean_not_alphanumeric_in_string(technology_clean, "_")[2]
                folder_name_clean = row_a.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")    
                folder_name_clean = fn_clean_accent_in_string(folder_name_clean)[2]
                file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(row_a.file_name)[2], "_")[2]
                file_name_clean   = fn_clean_accent_in_string(file_name_clean)[2]

                # Add trailing slash if "folder_name" is empty
                if folder_name_clean != "": folder_name_clean = f"{folder_name_clean}/"
                else:                       folder_name_clean = folder_name_clean

                # Generate source and destination file paths            
                src  = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/Excel/{folder_name_clean}{file_name_clean}.xlsx"
                dstn = f"{global_parameter.abfs_path_lh_bronze}/Files/processed/Excel/{folder_name_clean}{file_name_clean}/{now_year}/{now_month}/{file_name_clean}_{global_parameter.process_timestamp}.xlsx"
                if is_debug:
                    print(f"copy/delete file (src): {src}")
                    print(f"copy file (dstn): {dstn}")

                # Copy excel file to "processed"
                dict_parameter = {"technology": row_a.technology
, "technology_clean": technology_clean
, "frequency": row_a.frequency
, "folder_name": row_a.folder_name
, "folder_name_clean": folder_name_clean
, "file_name": row_a.file_name
, "file_name_clean": file_name_clean}
                result_copy = fn_copy_file(src, dstn, dict_parameter)[0]                
                if is_debug:
                    print(f"dict_parameter: {dict_parameter}")
                    print(f"result_copy: {result_copy}")

                # Delete excel file from "incoming"
                if result_copy == "Success": fn_delete_file(src, dict_parameter)

                if is_debug: print("-------------------- Loop Archive End --------------------")
            # Copy to lh_bronze/Files/processed (archive) - End

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)
            
            pdf_extracted = pdf_extracted.head(5)
            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)
            
            return (alert, alert_description)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_excel - End ----------\n")